In [4]:
%matplotlib qt
#ROTATION
import scipy.io as sio
import os
#from EEGs_persistence import *
from preprocess_data import *
from TDApipeline import *
from intensities_pipe import *
import time  
import sklearn.pipeline as skppl
import sklearn.linear_model as skllm
import sklearn.model_selection as skms
import sklearn.metrics as skm
import matplotlib.pyplot as plt
import sklearn.preprocessing as skprp
import pandas as pd
import sklearn.neighbors as sklnn
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D

'''
from joblib import Memory
from shutil import rmtree

from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
'''


def define_subject_dir(i_sub):
    """
    Creates the directory if it doesn't exist
    :param i_sub: subject id
    :return: directory path
    """
    res_dir = "results/subject_" + str(i_sub) + "/"
    if not os.path.exists(res_dir):
        print("create directory:", res_dir)
        os.makedirs(res_dir)
    return res_dir

def load_data(i_sub,space='both'):
    """
    Loads data from electrode space, font space 
    or both for a given subject
    :param i_sub: subject id
    :param space: electrode/font_space
    :return: data,directory path
    """
    subj_dir = define_subject_dir(i_sub)
    raw_data = sio.loadmat('data/dataClean-ICA3-'+str(i_sub)+'-T1.mat')
    
    if space=='electrodeSpace':
        elec_space=raw_data['dataSorted'] # [N,T,n_trials,motiv] 
        return elec_space,subj_dir
    elif space=='fontSpace':
        font_space=raw_data['ic_data3']
        return font_space,subj_dir
    else:
        elec_space=raw_data['dataSorted'] # [N,T,n_trials,motiv] 
        font_space=raw_data['ic_data3']
        
        return (elec_space,font_space),subj_dir
    


if __name__ == "__main__":
    subjects=[27]
    
    intensities=False
    exploratory=False
    classification=False
    PCA=True
    last=False
    
    
    
    bands=[-1,0,1,2]  
    #bands=[-1,2]
    n_band=len(bands)
    measures=["euclidean","correlation","quaf"]#,"dtw"]
    #measures=["quaf","dtw"]
    n_measure=len(measures)
    dimensions=["zero","one"]
    #dimensions=[]
    n_dim=len(dimensions)
    feat_vect=[DimensionLandScape(),DimensionSilhouette(),TopologicalDescriptors()]
    
    
    data_table=np.zeros((2*len(subjects),21))
    subj_t=0
    
    n_vectors=len(feat_vect)
    for subject in subjects:
        space='both'
        data_space,subj_dir=load_data(subject,space=space)

        spaces=['electrodeSpace','fontSpace']
        for sp in range(1):
            t=time.time()
            space=spaces[sp]
            if not os.path.exists(subj_dir+space):
                print("create directory(plot):",subj_dir+space)
                os.makedirs(subj_dir+'/'+space)
            print('cleaning and filtering data of',space,'of subject',subject)
            preprocessor=Preprocessor(data_space[sp])
            #filtered_ts_dic=preprocessor.get_filtered_ts_dic()
            ts_band,labels_original=preprocessor.get_trials_and_labels()
            
            data_table[subj_t,0]=preprocessor.N
            data_table[subj_t,1]=ts_band.shape[0]

            
            if intensities:
                for i_band in bands:
                    print('intensities for band ', i_band)
                    intensity(subj_dir,space,ts_band,labels,i_band)

            data_table[subj_t,6]=(labels_original==0).sum()
            data_table[subj_t,7]=(labels_original==1).sum()
            data_table[subj_t,8]=(labels_original==2).sum()
            

            vectors=[]
            sing_vals=[]
            
            blocs=[[0,11,8,9,4,5],[1,7,2,3,10,6]]
            band_dic={-1: 'noFilter', 0:'alpha',1:'betta',2:'gamma'}
            if PCA: 

                N=ts_band.shape[-1]
                persistence={}
                persistence_2d={}
                for i_band in bands:
                    pca_M0=[]
                    pca_M1=[]
                    pca_M2=[]
                    persistence[i_band]={}
                    persistence_2d[i_band]={}
                    print('global picture of band',band_dic[i_band] )
                    bloc_i=0
                    for bl in blocs:
                        #bloc=ts_band[:,i_band,:,:][temp]
                        PC=np.abs(ts_band[:,i_band,:,:]).mean(axis=1)
                        PC=PC.reshape((-1,N))
                        labels=labels_original
                        PC,labels=preprocessor.reject_outliers(PC,labels)
                        
                        tr2bl=preprocessor.tr2bl
                        temp=[tr_bl in bl for tr_bl in tr2bl]
                        PC=PC[temp]
                        labels=labels[temp]

                        data_table[subj_t,3+i_band]=PC.shape[0]

                        X =(PC - np.mean(PC, axis=0)).T
                        n = X.shape[1]
                        Y =  X.T/np.sqrt(n-1)
                        u, s, vh = la.svd(Y, full_matrices=False)
                        r=np.sum(np.where(s>1e-12,1,0))
                        #pca = vh[:r,:] @ X[:,:] # Principal components
                        variance_prop = s[:r]**2/np.sum(s[:r]**2) # Variance captured
                        acc_variance = np.cumsum(variance_prop)
                        std = s[:r]

                        '''
                        fig, axs = plt.subplots(1, 2, figsize=(18, 4))

                        # 3/4 of the total variance rule
                        axs[0].scatter(range(len(acc_variance)),acc_variance*100)
                        axs[0].set_xticks(range(len(acc_variance)), minor=False)
                        axs[0].hlines(75, xmin=0, xmax=len(std), colors='r', linestyles='dashdot')
                        axs[0].set_title('3/4 of the total variance rule')
                        axs[0].set_xlabel('PCA coordinates')
                        axs[0].set_ylabel('accumulated variance')
                        # Kraiser rule: Keep PC with eigenvalues > 1
                        # Scree plot: keep PCs before elbow
                        axs[1].scatter(range(len(std)),(std**2))
                        axs[1].set_xticks(range(len(acc_variance)), minor=False)

                        axs[1].hlines(1, xmin=0, xmax=len(std), colors='r', linestyles='dashdot')
                        axs[1].set_title('Scree Plot')
                        axs[1].set_xlabel('PCA coordinates')
                        axs[1].set_ylabel('eigenvalue')

                        if not os.path.exists(subj_dir+space+'/PCA/'+band_dic[i_band] ):
                            print("create directory(plot):",subj_dir+space+'/PCA/'+band_dic[i_band] )
                            os.makedirs(subj_dir+space+'/PCA/'+band_dic[i_band] )
                        plt.savefig(subj_dir+space+'/PCA/'+band_dic[i_band]+'/pca_plots.png')
                        plt.close()
                        print('acumulated variance:',acc_variance)'''
                        vectors.append(vh[:3,:])
                        sing_vals.append(s[:3])
                        #pca = vh[:3,:] @ X[:,:] 
                        pca = vectors[-1-bloc_i] @ X[:,:]
                        print(vectors[-1-bloc_i] )
                        pca=pca.T
                        
                        bloc_i=+1
                        #fig.add_subplot(projection='3d')
                        pca_M0.append(pca[labels==0])
                        pca_M1.append(pca[labels==1])
                        pca_M2.append(pca[labels==2])


                    fig = plt.figure(figsize=[18,8])
                    ax =fig.add_subplot(1, 1, 1, projection='3d')
                    fig.add_axes(ax)

                    ax.scatter(pca_M0[0][:,0],pca_M0[0][:,1],pca_M0[0][:,2],label='M0_0',c='r',alpha=0.5,zdir='z')
                    ax.scatter(pca_M1[0][:,0],pca_M1[0][:,1],pca_M1[0][:,2],label='M1_0',c='g',alpha=0.5,zdir='z')
                    ax.scatter(pca_M2[0][:,0],pca_M2[0][:,1],pca_M2[0][:,2],label='M2_0',c='b',alpha=0.5,zdir='z')
                    
                    
                    
                    angle3=np.arccos(np.dot(vectors[0][0],vectors[1][0]))
                    angle2=np.arccos(np.dot(vectors[0][1],vectors[1][1]))
                    angle1=np.arccos(np.dot(vectors[0][2],vectors[1][2]))
                    
                    rotmat1=np.array([[1,0,0],[0,np.cos(angle1),-np.sin(angle1)],[0,np.sin(angle1),np.cos(angle1)]])
                    rotmat2=np.array([[np.cos(angle2),0,np.sin(angle2)],[0,1,0],[-np.sin(angle2),0,np.cos(angle2)]])
                    rotmat3=np.array([[np.cos(angle3),-np.sin(angle3),0],[np.sin(angle3),np.cos(angle3),0],[0,0,1]])
                    rotmat=rotmat3 @ rotmat2 @ rotmat1
                    pca_M0[1]=pca_M0[1] @ rotmat
                    pca_M1[1]=pca_M1[1] @ rotmat
                    pca_M2[1]=pca_M2[1] @ rotmat
                
                    
                
                    
                    ax.scatter(pca_M0[1][:,0],pca_M0[1][:,1],pca_M0[1][:,2],label='M0_1',c='y',alpha=0.5,zdir='z')
                    ax.scatter(pca_M1[1][:,0],pca_M1[1][:,1],pca_M1[1][:,2],label='M1_1',c='k',alpha=0.5,zdir='z')
                    ax.scatter(pca_M2[1][:,0],pca_M2[1][:,1],pca_M2[1][:,2],label='M2_1',c='m',alpha=0.5,zdir='z')                    
                    
                    ax.legend()
                    ax.set_title(band_dic[i_band]+' pca projection PC of bloc')

                    ax.set_xlim3d(-1, 1)
                    ax.set_ylim3d(-1, 1)
                    ax.set_zlim3d(-1, 1)

                    ax.set_xlabel('$X$')
                    ax.set_ylabel('$Y$')
                    ax.set_zlabel('$Z$')
                    plt.show()
                    

                                 
                    





                    

        

                    
                    
                

                
                    

cleaning and filtering data of electrodeSpace of subject 27
there are 50 clean channels
global picture of band noFilter
[[ 1.98157082e-01  3.02311938e-01  3.52410092e-02  1.57955425e-01
   2.21661070e-01  1.69213057e-01  1.91162795e-01  6.94380745e-02
   8.51132978e-02  1.44273060e-01  4.85368779e-02  1.07440458e-01
   1.54570351e-01  1.96164612e-01  7.84854001e-02  7.95511608e-02
   1.36415874e-01  8.84352467e-02  7.05931465e-02  9.77214624e-02
   8.00106065e-02  1.69897497e-01  8.78473782e-02  8.44404626e-02
   1.09538088e-01  8.60387506e-02  8.41997325e-02  1.02086950e-01
   1.93349601e-01  1.21683988e-01  1.04900562e-01  1.08035328e-01
   1.17358367e-01  1.31430272e-01  1.21375446e-01  1.83060336e-01
   1.00966072e-01  1.07489932e-01  8.75800570e-02  8.93561735e-02
   1.00775281e-01  1.37864074e-01  2.00381725e-01  1.82152337e-01
   1.74622942e-01  1.38102004e-01  8.38926731e-02  1.98464625e-01
   1.84722391e-01  2.34607643e-01]
 [ 2.28470526e-01  5.89420884e-01  5.22105314e-01  2.

global picture of band betta
[[ 0.11597581  0.14124319  0.00698911  0.10070202  0.12727468  0.13546038
   0.19817893  0.09455411  0.10336571  0.13016514  0.03643118  0.08905648
   0.19107449  0.16145609  0.10547895  0.09763329  0.14088159  0.11161894
   0.09033858  0.09825711  0.09096582  0.17132699  0.11703464  0.08379268
   0.12045272  0.10695467  0.09207994  0.16302079  0.13813048  0.1135431
   0.12046322  0.12455748  0.14676961  0.13584268  0.14595697  0.20003432
   0.16580128  0.17247582  0.13015848  0.13688685  0.12808205  0.13846102
   0.19632758  0.24502515  0.16775476  0.15329867  0.10144207  0.22945751
   0.19128279  0.20614756]
 [ 0.09751863  0.12055974  0.97654282  0.05111378  0.06391315  0.00409296
  -0.02701259  0.03000357  0.01459612  0.02807498 -0.04975703  0.01457541
  -0.01351103 -0.00774944  0.00937582  0.0098782  -0.04732884  0.00689026
   0.0087172   0.00586039 -0.0081947  -0.01161199  0.02422716  0.02076548
  -0.00311797  0.0049283   0.02191367  0.01121861 -0.0080

In [11]:
%matplotlib qt
#Nothing
import scipy.io as sio
import os
#from EEGs_persistence import *
from preprocess_data import *
from TDApipeline import *
from intensities_pipe import *
import time  
import sklearn.pipeline as skppl
import sklearn.linear_model as skllm
import sklearn.model_selection as skms
import sklearn.metrics as skm
import matplotlib.pyplot as plt
import sklearn.preprocessing as skprp
import pandas as pd
import sklearn.neighbors as sklnn
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D

'''
from joblib import Memory
from shutil import rmtree

from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
'''


def define_subject_dir(i_sub):
    """
    Creates the directory if it doesn't exist
    :param i_sub: subject id
    :return: directory path
    """
    res_dir = "results/subject_" + str(i_sub) + "/"
    if not os.path.exists(res_dir):
        print("create directory:", res_dir)
        os.makedirs(res_dir)
    return res_dir

def load_data(i_sub,space='both'):
    """
    Loads data from electrode space, font space 
    or both for a given subject
    :param i_sub: subject id
    :param space: electrode/font_space
    :return: data,directory path
    """
    subj_dir = define_subject_dir(i_sub)
    raw_data = sio.loadmat('data/dataClean-ICA3-'+str(i_sub)+'-T1.mat')
    
    if space=='electrodeSpace':
        elec_space=raw_data['dataSorted'] # [N,T,n_trials,motiv] 
        return elec_space,subj_dir
    elif space=='fontSpace':
        font_space=raw_data['ic_data3']
        return font_space,subj_dir
    else:
        elec_space=raw_data['dataSorted'] # [N,T,n_trials,motiv] 
        font_space=raw_data['ic_data3']
        return (elec_space,font_space),subj_dir,raw_data['indexM']
    


if __name__ == "__main__":
    subjects=[25]
    
    intensities=False
    exploratory=False
    classification=False
    PCA=True
    last=False
    
         
    bloc_dic={}
    bloc_subj_dic={}
    bloc_subj_dic[27]=np.array([[1, 2, 10, 6, 7, 3],[5, 2, 4, 1, 8, 9]])
    bloc_subj_dic[25]=np.array([[1, 2, 8, 3, 5, 4],[6, 7, 2, 10, 1, 9]])
    
    bands=[-1,0,1,2]  
    #bands=[-1,2]
    n_band=len(bands)
    measures=["euclidean","correlation","quaf"]#,"dtw"]
    #measures=["quaf","dtw"]
    n_measure=len(measures)
    dimensions=["zero","one"]
    #dimensions=[]
    n_dim=len(dimensions)
    feat_vect=[DimensionLandScape(),DimensionSilhouette(),TopologicalDescriptors()]
    
    
    data_table=np.zeros((2*len(subjects),21))
    subj_t=0
    
    n_vectors=len(feat_vect)
    for subject in subjects:
        space='both'
        data_space,subj_dir,index=load_data(subject,space=space)
        index=index[0]

        cont1=0
        cont2=0
        for ind in range(12):
            if index[ind]==1:
                cont1+=1
                if cont1==2:
                    index[ind]=11
            if index[ind]==2:
                cont2+=1
                if cont2==2:
                    index[ind]=12 
        
        bloc_subj_dic[subject][1][bloc_subj_dic[subject][1]<=2]=bloc_subj_dic[subject][1][bloc_subj_dic[subject][1]<=2]+10

        bloc_session=np.where([ind in bloc_subj_dic[subject][1] for ind in index],2,1 )
        for sp in range(1):
            t=time.time()
            space=spaces[sp]
            if not os.path.exists(subj_dir+space):
                print("create directory(plot):",subj_dir+space)
                os.makedirs(subj_dir+'/'+space)
            print('cleaning and filtering data of',space,'of subject',subject)
            preprocessor=Preprocessor(data_space[sp])
            #filtered_ts_dic=preprocessor.get_filtered_ts_dic()
            ts_band,labels_original=preprocessor.get_trials_and_labels()
            
            data_table[subj_t,0]=preprocessor.N
            data_table[subj_t,1]=ts_band.shape[0]

            
            if intensities:
                for i_band in bands:
                    print('intensities for band ', i_band)
                    intensity(subj_dir,space,ts_band,labels,i_band)

            data_table[subj_t,6]=(labels_original==0).sum()
            data_table[subj_t,7]=(labels_original==1).sum()
            data_table[subj_t,8]=(labels_original==2).sum()
            

            vectors=[]
            vectors3d=[]
            sing_vals=[]
            
            blocs=[]
            blocs.append(np.array(list(range(12)))[bloc_session==1])
            blocs.append(np.array(list(range(12)))[bloc_session==2])
            if PCA: 

                N=ts_band.shape[-1]
                persistence={}
                persistence_2d={}
                for i_band in bands:
                    pca_M0=[]
                    pca_M1=[]
                    pca_M2=[]
                    persistence[i_band]={}
                    persistence_2d[i_band]={}
                    print('global picture of band',band_dic[i_band] )
                    bloc_i=0
                    for bl in blocs:
                        #bloc=ts_band[:,i_band,:,:][temp]
                        PC=np.abs(ts_band[:,i_band,:,:]).mean(axis=1)
                        PC=PC.reshape((-1,N))
                        labels=labels_original
                        PC,labels=preprocessor.reject_outliers(PC,labels)
                        
                        tr2bl=preprocessor.tr2bl
                        temp=[tr_bl in bl for tr_bl in tr2bl]
                        PC=PC[temp]
                        labels=labels[temp]

                        data_table[subj_t,3+i_band]=PC.shape[0]

                        X =(PC - np.mean(PC, axis=0)).T
                        n = X.shape[1]
                        Y =  X.T/np.sqrt(n-1)
                        u, s, vh = la.svd(Y, full_matrices=False)
                        r=np.sum(np.where(s>1e-12,1,0))
                        #pca = vh[:r,:] @ X[:,:] # Principal components
                        variance_prop = s[:r]**2/np.sum(s[:r]**2) # Variance captured
                        acc_variance = np.cumsum(variance_prop)
                        std = s[:r]

                        '''
                        fig, axs = plt.subplots(1, 2, figsize=(18, 4))

                        # 3/4 of the total variance rule
                        axs[0].scatter(range(len(acc_variance)),acc_variance*100)
                        axs[0].set_xticks(range(len(acc_variance)), minor=False)
                        axs[0].hlines(75, xmin=0, xmax=len(std), colors='r', linestyles='dashdot')
                        axs[0].set_title('3/4 of the total variance rule')
                        axs[0].set_xlabel('PCA coordinates')
                        axs[0].set_ylabel('accumulated variance')
                        # Kraiser rule: Keep PC with eigenvalues > 1
                        # Scree plot: keep PCs before elbow
                        axs[1].scatter(range(len(std)),(std**2))
                        axs[1].set_xticks(range(len(acc_variance)), minor=False)

                        axs[1].hlines(1, xmin=0, xmax=len(std), colors='r', linestyles='dashdot')
                        axs[1].set_title('Scree Plot')
                        axs[1].set_xlabel('PCA coordinates')
                        axs[1].set_ylabel('eigenvalue')

                        if not os.path.exists(subj_dir+space+'/PCA/'+band_dic[i_band] ):
                            print("create directory(plot):",subj_dir+space+'/PCA/'+band_dic[i_band] )
                            os.makedirs(subj_dir+space+'/PCA/'+band_dic[i_band] )
                        plt.savefig(subj_dir+space+'/PCA/'+band_dic[i_band]+'/pca_plots.png')
                        plt.close()
                        print('acumulated variance:',acc_variance)'''
                        vectors.append(vh[:3,:])
                        sing_vals.append(s[:3])
                        pca = vh[:3,:] @ X[:,:] 


                        pca=pca.T

                        u2, s2, vh2=la.svd(pca, full_matrices=False)
                        vectors3d.append(vh2)
                        
                        bloc_i=+1
                        #fig.add_subplot(projection='3d')
                        pca_M0.append(pca[labels==0])
                        pca_M1.append(pca[labels==1])
                        pca_M2.append(pca[labels==2])

                    

                    
                    fig = plt.figure(figsize=[18,8])
                    ax =fig.add_subplot(1, 1, 1, projection='3d')
                    fig.add_axes(ax)
                    '''
                    pca_M0[0]=pca_M0[0] / pca_M0[0].std()
                    pca_M1[0]=pca_M1[0] / pca_M1[0].std()
                    pca_M2[0]=pca_M2[0] / pca_M2[0].std()'''
                    
                    
                    ax.scatter(pca_M0[0][:,0],pca_M0[0][:,1],pca_M0[0][:,2],label='M0_0',c='r',alpha=0.5,zdir='z')
                    ax.scatter(pca_M1[0][:,0],pca_M1[0][:,1],pca_M1[0][:,2],label='M1_0',c='g',alpha=0.5,zdir='z')
                    ax.scatter(pca_M2[0][:,0],pca_M2[0][:,1],pca_M2[0][:,2],label='M2_0',c='b',alpha=0.5,zdir='z')
                    
                    
                    '''
                    angle3=np.arccos(np.dot(vectors[0][0],vectors[1][0]))
                    angle2=np.arccos(np.dot(vectors[0][1],vectors[1][1]))
                    angle1=np.arccos(np.dot(vectors[0][2],vectors[1][2]))
                    
                    rotmat1=np.array([[1,0,0],[0,np.cos(angle1),-np.sin(angle1)],[0,np.sin(angle1),np.cos(angle1)]])
                    rotmat2=np.array([[np.cos(angle2),0,np.sin(angle2)],[0,1,0],[-np.sin(angle2),0,np.cos(angle2)]])
                    rotmat3=np.array([[np.cos(angle3),-np.sin(angle3),0],[np.sin(angle3),np.cos(angle3),0],[0,0,1]])
                    rotmat=rotmat3 @ rotmat2 @ rotmat1
                    
                    
                    pca_M0[1]=pca_M0[1] @ rotmat
                    pca_M1[1]=pca_M1[1] @ rotmat
                    pca_M2[1]=pca_M2[1] @ rotmat'''
                
                    '''
                    pca_M0[1]=pca_M0[1] / pca_M0[1].std()
                    pca_M1[1]=pca_M1[1] / pca_M1[1].std()
                    pca_M2[1]=pca_M2[1] / pca_M2[1].std()'''
                
                    
                    ax.scatter(pca_M0[1][:,0],pca_M0[1][:,1],pca_M0[1][:,2],label='M0_1',c='y',alpha=0.5,zdir='z')
                    ax.scatter(pca_M1[1][:,0],pca_M1[1][:,1],pca_M1[1][:,2],label='M1_1',c='k',alpha=0.5,zdir='z')
                    ax.scatter(pca_M2[1][:,0],pca_M2[1][:,1],pca_M2[1][:,2],label='M2_1',c='m',alpha=0.5,zdir='z')                    
                    
                    ax.legend()
                    ax.set_title(band_dic[i_band]+' pca projection PC')

                    ax.set_xlim3d(-1, 1)
                    ax.set_ylim3d(-1, 1)
                    ax.set_zlim3d(-1, 1)

                    ax.set_xlabel('$X$')
                    ax.set_ylabel('$Y$')
                    ax.set_zlabel('$Z$')
                    plt.show()
                    

                                 
                    





                    

        

                    
                    
                

                
                    

cleaning and filtering data of electrodeSpace of subject 25
there are 42 clean channels
global picture of band noFilter
global picture of band alpha
global picture of band betta
global picture of band gamma


In [3]:
vectors3d

[array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00, -1.00000000e+00,  1.83880688e-16],
        [-0.00000000e+00, -1.80411242e-16, -1.00000000e+00]]),
 array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-0.00000000e+00,  1.00000000e+00, -7.94503352e-16],
        [ 0.00000000e+00,  7.92768629e-16,  1.00000000e+00]]),
 array([[-1.00000000e+00, -0.00000000e+00,  0.00000000e+00],
        [-0.00000000e+00,  1.00000000e+00, -9.43689571e-15],
        [-0.00000000e+00, -9.45077350e-15, -1.00000000e+00]]),
 array([[-1.00000000e+00, -0.00000000e+00, -0.00000000e+00],
        [ 0.00000000e+00, -1.00000000e+00,  5.61356517e-15],
        [ 0.00000000e+00,  5.61356517e-15,  1.00000000e+00]]),
 array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-0.00000000e+00,  1.00000000e+00,  5.21804822e-15],
        [ 0.00000000e+00, -5.20417043e-15,  1.00000000e+00]]),
 array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00

In [5]:
np.concatenate((pca_M0[0],pca_M1[0],pca_M2[0]),axis=0).mean()

3.2611302667175225e-17

In [6]:
np.concatenate((pca_M0[1],pca_M1[1],pca_M2[1]),axis=0).mean()

-2.1201189899065222e-17

In [7]:
np.concatenate((pca_M0[0],pca_M1[0],pca_M2[0]),axis=0).std()

0.1001134400645769

In [8]:
np.concatenate((pca_M0[1],pca_M1[1],pca_M2[1]),axis=0).std()

0.17194403358740804

In [9]:
vectors[0].T @rotmat

array([[-2.77772183e-02,  1.75438641e-01,  4.27213248e-05],
       [-1.74481880e-01,  7.53075907e-01, -5.49779619e-02],
       [-5.40699511e-02,  2.69472382e-02,  9.93066335e-01],
       [-5.20079924e-02,  1.79506682e-01, -7.19972340e-03],
       [-5.27574594e-02,  2.49488585e-01, -7.88922843e-03],
       [ 7.88929768e-04,  1.50595043e-01,  1.81909358e-03],
       [ 2.32027638e-02,  4.55661625e-02,  1.27370036e-02],
       [-6.21926732e-02,  1.90859902e-01, -6.74838527e-03],
       [ 2.18358029e-03,  1.07027223e-01,  1.38087990e-02],
       [-3.02297924e-02,  1.30844067e-01, -9.26774307e-04],
       [ 6.12383408e-02,  3.17680343e-02,  2.24802181e-02],
       [ 1.01263606e-02,  1.03471829e-01,  8.89958911e-03],
       [ 1.34704285e-02,  3.72077045e-02,  7.61183880e-03],
       [ 2.26582957e-02,  5.20005762e-02,  1.37889428e-02],
       [ 2.11953278e-02,  3.98121918e-02,  1.41481110e-02],
       [-1.09545830e-02,  9.00138146e-02,  3.36277155e-03],
       [ 5.58048848e-02,  4.27380497e-02

In [10]:
vectors[1].T 

array([[ 3.11334229e-01, -1.47475786e-01, -1.25472978e-01],
       [ 5.31836052e-01, -3.24335008e-01, -2.35380325e-01],
       [ 1.29080017e-01, -1.92810409e-02, -4.00756445e-02],
       [ 2.36313592e-01, -1.27547454e-01, -1.21405048e-01],
       [ 3.84970891e-01, -1.01100584e-01, -9.08719338e-02],
       [ 1.69508893e-01,  1.48002234e-01,  9.97917364e-02],
       [-6.80827497e-03,  1.02953796e-01,  3.22605211e-02],
       [ 1.14172731e-01,  2.12719012e-02, -2.41401311e-03],
       [ 8.53847364e-02,  3.65854834e-02,  3.26478917e-03],
       [ 1.78594397e-01, -7.53631171e-02, -5.66143544e-02],
       [ 6.22734796e-02,  7.26307717e-02,  1.96075552e-02],
       [ 1.82679065e-01,  6.88021238e-02,  2.42869101e-02],
       [ 3.86202414e-02,  7.01490501e-02,  7.28691223e-02],
       [ 2.05588522e-01,  4.63156235e-01,  4.92846952e-01],
       [ 6.36847687e-02,  6.28437942e-02,  1.14513345e-02],
       [ 9.25477860e-02,  7.02979135e-02, -6.72228178e-03],
       [ 9.49521939e-02,  9.06365979e-02

In [11]:
vectors[0].T

array([[ 1.67330145e-02,  1.76805131e-01, -3.20067154e-03],
       [ 1.89264835e-02,  7.73619392e-01,  4.17689989e-02],
       [ 9.92804357e-01, -5.79713501e-02,  2.84480074e-02],
       [ 1.07667425e-02,  1.85633129e-01,  2.00931566e-02],
       [ 1.63635882e-02,  2.54455070e-01,  8.65989138e-03],
       [ 1.52442767e-02,  1.47375479e-01, -2.70347640e-02],
       [ 1.59094799e-02,  3.95042541e-02, -3.10350935e-02],
       [ 1.26002342e-02,  1.98470832e-01,  2.81342208e-02],
       [ 2.32327863e-02,  1.03280213e-01, -2.10644867e-02],
       [ 1.18705009e-02,  1.33585122e-01,  6.99824417e-03],
       [ 2.29944127e-02,  1.85309981e-02, -6.62764037e-02],
       [ 1.77405959e-02,  9.88944807e-02, -2.81659037e-02],
       [ 1.04139698e-02,  3.34599494e-02, -1.98943918e-02],
       [ 1.75514553e-02,  4.58083701e-02, -3.16406839e-02],
       [ 1.68724361e-02,  3.40727389e-02, -2.80854222e-02],
       [ 1.17910440e-02,  8.98343295e-02, -4.95605316e-03],
       [ 1.38824295e-02,  3.11957753e-02

In [12]:
vectors[1].T @rotmat

array([[ 1.37531282e-01, -9.49450045e-02,  3.26331382e-01],
       [ 2.68071160e-01, -2.29535249e-01,  5.64710637e-01],
       [ 3.80806689e-02, -3.91283360e-04,  1.31107499e-01],
       [ 1.32829334e-01, -8.28167947e-02,  2.49698617e-01],
       [ 9.28423171e-02, -4.89087336e-02,  3.94550203e-01],
       [-1.29775846e-01,  1.42917269e-01,  1.52738997e-01],
       [-4.91703693e-02,  9.47368100e-02, -1.71402879e-02],
       [-5.41192838e-03,  3.14871462e-02,  1.11682605e-01],
       [-1.25850314e-02,  4.29422960e-02,  8.14695395e-02],
       [ 6.21855524e-02, -4.80827048e-02,  1.86015466e-01],
       [-3.40182767e-02,  7.33788591e-02,  5.47354671e-02],
       [-4.23351195e-02,  7.95730287e-02,  1.74845926e-01],
       [-8.51717961e-02,  5.95569068e-02,  3.03489909e-02],
       [-5.72101708e-01,  3.86763310e-01,  1.50970970e-01],
       [-2.43610847e-02,  6.53274749e-02,  5.72283484e-02],
       [-8.79393274e-03,  7.83819467e-02,  8.56213500e-02],
       [-5.17750113e-02,  9.15730403e-02

In [7]:
%matplotlib qt

import scipy.io as sio
import os
#from EEGs_persistence import *
from preprocess_data import *
from TDApipeline import *
from intensities_pipe import *
import time  
import sklearn.pipeline as skppl
import sklearn.linear_model as skllm
import sklearn.model_selection as skms
import sklearn.metrics as skm
import matplotlib.pyplot as plt
import sklearn.preprocessing as skprp
import pandas as pd
import sklearn.neighbors as sklnn
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D
'''
from joblib import Memory
from shutil import rmtree

from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
'''
#projection

def define_subject_dir(i_sub):
    """
    Creates the directory if it doesn't exist
    :param i_sub: subject id
    :return: directory path
    """
    res_dir = "results/subject_" + str(i_sub) + "/"
    if not os.path.exists(res_dir):
        print("create directory:", res_dir)
        os.makedirs(res_dir)
    return res_dir

def load_data(i_sub,space='both'):
    """
    Loads data from electrode space, font space 
    or both for a given subject
    :param i_sub: subject id
    :param space: electrode/font_space
    :return: data,directory path
    """
    subj_dir = define_subject_dir(i_sub)
    raw_data = sio.loadmat('data/dataClean-ICA3-'+str(i_sub)+'-T1.mat')
    
    if space=='electrodeSpace':
        elec_space=raw_data['dataSorted'] # [N,T,n_trials,motiv] 
        return elec_space,subj_dir
    elif space=='fontSpace':
        font_space=raw_data['ic_data3']
        return font_space,subj_dir
    else:
        elec_space=raw_data['dataSorted'] # [N,T,n_trials,motiv] 
        font_space=raw_data['ic_data3']
        return (elec_space,font_space),subj_dir,raw_data['indexM']
    
    


if __name__ == "__main__":
    subjects=[25]
    
    intensities=False
    exploratory=False
    classification=False
    PCA=True
    last=False
     
    bloc_dic={}
    bloc_subj_dic={}
    bloc_subj_dic[27]=np.array([[1, 2, 10, 6, 7, 3],[5, 2, 4, 1, 8, 9]])
    bloc_subj_dic[25]=np.array([[1, 2, 8, 3, 5, 4],[6, 7, 2, 10, 1, 9]])

    
    bands=[-1,0,1,2]  
    #bands=[-1,2]
    n_band=len(bands)
    measures=["euclidean","correlation","quaf"]#,"dtw"]
    #measures=["quaf","dtw"]
    n_measure=len(measures)
    dimensions=["zero","one"]
    #dimensions=[]
    n_dim=len(dimensions)
    feat_vect=[DimensionLandScape(),DimensionSilhouette(),TopologicalDescriptors()]
    
    
    data_table=np.zeros((2*len(subjects),21))
    subj_t=0
    
    n_vectors=len(feat_vect)
    for subject in subjects:
        space='both'
        data_space,subj_dir,index=load_data(subject,space=space)

        spaces=['electrodeSpace','fontSpace']
        index=index[0]

        cont1=0
        cont2=0
        for ind in range(12):
            if index[ind]==1:
                cont1+=1
                if cont1==2:
                    index[ind]=11
            if index[ind]==2:
                cont2+=1
                if cont2==2:
                    index[ind]=12 
        
        bloc_subj_dic[subject][1][bloc_subj_dic[subject][1]<=2]=bloc_subj_dic[subject][1][bloc_subj_dic[subject][1]<=2]+10

        bloc_session=np.where([ind in bloc_subj_dic[subject][1] for ind in index],2,1 )
        for sp in range(1):

            space=spaces[sp]
            if not os.path.exists(subj_dir+space):
                print("create directory(plot):",subj_dir+space)
                os.makedirs(subj_dir+'/'+space)
            print('cleaning and filtering data of',space,'of subject',subject)
            preprocessor=Preprocessor(data_space[sp])
            #filtered_ts_dic=preprocessor.get_filtered_ts_dic()
            ts_band,labels_original=preprocessor.get_trials_and_labels()
            
            data_table[subj_t,0]=preprocessor.N
            data_table[subj_t,1]=ts_band.shape[0]

            
            if intensities:
                for i_band in bands:
                    print('intensities for band ', i_band)
                    intensity(subj_dir,space,ts_band,labels,i_band)

            data_table[subj_t,6]=(labels_original==0).sum()
            data_table[subj_t,7]=(labels_original==1).sum()
            data_table[subj_t,8]=(labels_original==2).sum()
            

            vectors=[]
            sing_vals=[]
            
            blocs=[]
            blocs.append(np.array(list(range(12)))[bloc_session==1])
            blocs.append(np.array(list(range(12)))[bloc_session==2])

            band_dic={-1: 'noFilter', 0:'alpha',1:'betta',2:'gamma'}
            if PCA: 

                N=ts_band.shape[-1]
                persistence={}
                persistence_2d={}
                for i_band in bands:
                    pca_M0=[]
                    pca_M1=[]
                    pca_M2=[]
                    persistence[i_band]={}
                    persistence_2d[i_band]={}
                    print('global picture of band',band_dic[i_band] )
                    bloc_i=0
                    for bl in blocs:
                        #bloc=ts_band[:,i_band,:,:][temp]
                        PC=np.abs(ts_band[:,i_band,:,:]).mean(axis=1)
                        PC=PC.reshape((-1,N))
                        labels=labels_original
                        PC,labels=preprocessor.reject_outliers(PC,labels)
                        
                        tr2bl=preprocessor.tr2bl
                        temp=[tr_bl in bl for tr_bl in tr2bl]
                        PC=PC[temp]
                        labels=labels[temp]

                        data_table[subj_t,3+i_band]=PC.shape[0]

                        X =(PC - np.mean(PC, axis=0)).T
                        n = X.shape[1]
                        Y =  X.T/np.sqrt(n-1)
                        
                        u, s, vh = la.svd(Y, full_matrices=False)
                        r=np.sum(np.where(s>1e-12,1,0))
                        #pca = vh[:r,:] @ X[:,:] # Principal components
                        variance_prop = s[:r]**2/np.sum(s[:r]**2) # Variance captured
                        acc_variance = np.cumsum(variance_prop)
                        std = s[:r]

                        '''
                        fig, axs = plt.subplots(1, 2, figsize=(18, 4))

                        # 3/4 of the total variance rule
                        axs[0].scatter(range(len(acc_variance)),acc_variance*100)
                        axs[0].set_xticks(range(len(acc_variance)), minor=False)
                        axs[0].hlines(75, xmin=0, xmax=len(std), colors='r', linestyles='dashdot')
                        axs[0].set_title('3/4 of the total variance rule')
                        axs[0].set_xlabel('PCA coordinates')
                        axs[0].set_ylabel('accumulated variance')
                        # Kraiser rule: Keep PC with eigenvalues > 1
                        # Scree plot: keep PCs before elbow
                        axs[1].scatter(range(len(std)),(std**2))
                        axs[1].set_xticks(range(len(acc_variance)), minor=False)

                        axs[1].hlines(1, xmin=0, xmax=len(std), colors='r', linestyles='dashdot')
                        axs[1].set_title('Scree Plot')
                        axs[1].set_xlabel('PCA coordinates')
                        axs[1].set_ylabel('eigenvalue')

                        if not os.path.exists(subj_dir+space+'/PCA/'+band_dic[i_band] ):
                            print("create directory(plot):",subj_dir+space+'/PCA/'+band_dic[i_band] )
                            os.makedirs(subj_dir+space+'/PCA/'+band_dic[i_band] )
                        plt.savefig(subj_dir+space+'/PCA/'+band_dic[i_band]+'/pca_plots.png')
                        plt.close()
                        print('acumulated variance:',acc_variance)'''
                        vectors.append(vh[:3,:])
                        sing_vals.append(s[:3])
                        print(bloc_i)
                        if bloc_i==1:
                            proj=np.zeros((3,X.shape[0]))
                            for dim in range(3):
                                proj[dim]=(np.dot(vh[dim,:],vectors[-2][dim])*vectors[-2][dim]) / np.linalg.norm(vectors[-2][dim])**2
                        
                            pca = proj @ X[:,:] 
                        else:
                            pca = vh[:3,:]  @ X[:,:] 
                        #pca = vectors[0] @ X[:,:] 
                        pca=pca.T
                        bloc_i+=1
                        #fig.add_subplot(projection='3d')
                        pca_M0.append(pca[labels==0])
                        pca_M1.append(pca[labels==1])
                        pca_M2.append(pca[labels==2])


                    fig = plt.figure(figsize=[18,8])
                    ax =fig.add_subplot(1, 1, 1, projection='3d')
                    fig.add_axes(ax)

                    ax.scatter(pca_M0[0][:,0],pca_M0[0][:,1],pca_M0[0][:,2],label='M0_0',c='r',alpha=0.5,zdir='z')
                    ax.scatter(pca_M1[0][:,0],pca_M1[0][:,1],pca_M1[0][:,2],label='M1_0',c='g',alpha=0.5,zdir='z')
                    ax.scatter(pca_M2[0][:,0],pca_M2[0][:,1],pca_M2[0][:,2],label='M2_0',c='b',alpha=0.5,zdir='z')
                    
                    
                    
                    ax.scatter(pca_M0[1][:,0],pca_M0[1][:,1],pca_M0[1][:,2],label='M0_1',c='y',alpha=0.5,zdir='z')
                    ax.scatter(pca_M1[1][:,0],pca_M1[1][:,1],pca_M1[1][:,2],label='M1_1',c='k',alpha=0.5,zdir='z')
                    ax.scatter(pca_M2[1][:,0],pca_M2[1][:,1],pca_M2[1][:,2],label='M2_1',c='m',alpha=0.5,zdir='z')                    
                    
                    ax.legend()
                    ax.set_title(band_dic[i_band]+' pca projection PC')

                    ax.set_xlim3d(-1, 1)
                    ax.set_ylim3d(-1, 1)
                    ax.set_zlim3d(-1, 1)

                    ax.set_xlabel('$X$')
                    ax.set_ylabel('$Y$')
                    ax.set_zlabel('$Z$')
                    plt.show()
                    

                                 
                    





                    

        

                    
                    
                

                
                    

> <ipython-input-7-db3fdaace092>(76)<module>()
     74 
     75     breakpoint()
---> 76     bloc_dic={}
     77     bloc_subj_dic={}
     78     bloc_subj_dic[27]=np.array([[1, 2, 10, 6, 7, 3],[5, 2, 4, 1, 8, 9]])

ipdb> n
> <ipython-input-7-db3fdaace092>(77)<module>()
     75     breakpoint()
     76     bloc_dic={}
---> 77     bloc_subj_dic={}
     78     bloc_subj_dic[27]=np.array([[1, 2, 10, 6, 7, 3],[5, 2, 4, 1, 8, 9]])
     79     bloc_subj_dic[25]=np.array([[1, 2, 8, 3, 5, 4],[6, 7, 2, 10, 1, 9]])

ipdb> n
> <ipython-input-7-db3fdaace092>(78)<module>()
     76     bloc_dic={}
     77     bloc_subj_dic={}
---> 78     bloc_subj_dic[27]=np.array([[1, 2, 10, 6, 7, 3],[5, 2, 4, 1, 8, 9]])
     79     bloc_subj_dic[25]=np.array([[1, 2, 8, 3, 5, 4],[6, 7, 2, 10, 1, 9]])
     80 

ipdb> n
> <ipython-input-7-db3fdaace092>(79)<module>()
     77     bloc_subj_dic={}
     78     bloc_subj_dic[27]=np.array([[1, 2, 10, 6, 7, 3],[5, 2, 4, 1, 8, 9]])
---> 79     bloc_subj_dic[25]=np.array([[1,

ipdb> n
> <ipython-input-7-db3fdaace092>(88)<module>()
     86     #measures=["quaf","dtw"]
     87     n_measure=len(measures)
---> 88     dimensions=["zero","one"]
     89     #dimensions=[]
     90     n_dim=len(dimensions)

ipdb> n
> <ipython-input-7-db3fdaace092>(90)<module>()
     88     dimensions=["zero","one"]
     89     #dimensions=[]
---> 90     n_dim=len(dimensions)
     91     feat_vect=[DimensionLandScape(),DimensionSilhouette(),TopologicalDescriptors()]
     92 

ipdb> n
> <ipython-input-7-db3fdaace092>(91)<module>()
     89     #dimensions=[]
     90     n_dim=len(dimensions)
---> 91     feat_vect=[DimensionLandScape(),DimensionSilhouette(),TopologicalDescriptors()]
     92 
     93 

ipdb> n
> <ipython-input-7-db3fdaace092>(94)<module>()
     92 
     93 
---> 94     data_table=np.zeros((2*len(subjects),21))
     95     subj_t=0
     96 

ipdb> n
> <ipython-input-7-db3fdaace092>(95)<module>()
     93 
     94     data_table=np.zeros((2*len(subjects),21))
---> 95     s

ipdb> n
> <ipython-input-7-db3fdaace092>(106)<module>()
    104 
    105         cont1=0
--> 106         cont2=0
    107         for ind in range(12):
    108             if index[ind]==1:

ipdb> n
> <ipython-input-7-db3fdaace092>(107)<module>()
    105         cont1=0
    106         cont2=0
--> 107         for ind in range(12):
    108             if index[ind]==1:
    109                 cont1+=1

ipdb> n
> <ipython-input-7-db3fdaace092>(108)<module>()
    106         cont2=0
    107         for ind in range(12):
--> 108             if index[ind]==1:
    109                 cont1+=1
    110                 if cont1==2:

ipdb> n
> <ipython-input-7-db3fdaace092>(112)<module>()
    110                 if cont1==2:
    111                     index[ind]=11
--> 112             if index[ind]==2:
    113                 cont2+=1
    114                 if cont2==2:

ipdb> n
> <ipython-input-7-db3fdaace092>(113)<module>()
    111                     index[ind]=11
    112             if inde

ipdb> n
> <ipython-input-7-db3fdaace092>(107)<module>()
    105         cont1=0
    106         cont2=0
--> 107         for ind in range(12):
    108             if index[ind]==1:
    109                 cont1+=1

ipdb> n
> <ipython-input-7-db3fdaace092>(108)<module>()
    106         cont2=0
    107         for ind in range(12):
--> 108             if index[ind]==1:
    109                 cont1+=1
    110                 if cont1==2:

ipdb> n
> <ipython-input-7-db3fdaace092>(112)<module>()
    110                 if cont1==2:
    111                     index[ind]=11
--> 112             if index[ind]==2:
    113                 cont2+=1
    114                 if cont2==2:

ipdb> n
> <ipython-input-7-db3fdaace092>(107)<module>()
    105         cont1=0
    106         cont2=0
--> 107         for ind in range(12):
    108             if index[ind]==1:
    109                 cont1+=1

ipdb> n
> <ipython-input-7-db3fdaace092>(108)<module>()
    106         cont2=0
    107         for i

*** NameError: name 'inf' is not defined
ipdb> ind
5
ipdb> n
> <ipython-input-7-db3fdaace092>(112)<module>()
    110                 if cont1==2:
    111                     index[ind]=11
--> 112             if index[ind]==2:
    113                 cont2+=1
    114                 if cont2==2:

ipdb> n
> <ipython-input-7-db3fdaace092>(107)<module>()
    105         cont1=0
    106         cont2=0
--> 107         for ind in range(12):
    108             if index[ind]==1:
    109                 cont1+=1

ipdb> n
> <ipython-input-7-db3fdaace092>(108)<module>()
    106         cont2=0
    107         for ind in range(12):
--> 108             if index[ind]==1:
    109                 cont1+=1
    110                 if cont1==2:

ipdb> n
> <ipython-input-7-db3fdaace092>(109)<module>()
    107         for ind in range(12):
    108             if index[ind]==1:
--> 109                 cont1+=1
    110                 if cont1==2:
    111                     index[ind]=11

ipdb> n
> <ipytho

...

ipdb> n
> <ipython-input-7-db3fdaace092>(223)<module>()
    221                             proj=np.zeros((3,X.shape[0]))
    222                             for dim in range(3):
--> 223                                 proj[dim]=(np.dot(vh[dim,:],vectors[-2][dim])*vectors[-2][dim]) / np.linalg.norm(vectors[-2][dim])**2
    224 
    225                             pca = proj @ X[:,:]

ipdb> dim
1
ipdb> n
> <ipython-input-7-db3fdaace092>(222)<module>()
    220                         if bloc_i==1:
    221                             proj=np.zeros((3,X.shape[0]))
--> 222                             for dim in range(3):
    223                                 proj[dim]=(np.dot(vh[dim,:],vectors[-2][dim])*vectors[-2][dim]) / np.linalg.norm(vectors[-2][dim])**2
    224 

ipdb> n
> <ipython-input-7-db3fdaace092>(223)<module>()
    221                             proj=np.zeros((3,X.shape[0]))
    222                             for dim in range(3):
--> 223                                 proj[

ipdb> n
> <ipython-input-7-db3fdaace092>(230)<module>()
    228                         #pca = vectors[0] @ X[:,:]
    229                         pca=pca.T
--> 230                         bloc_i+=1
    231                         #fig.add_subplot(projection='3d')
    232                         pca_M0.append(pca[labels==0])

ipdb> n
> <ipython-input-7-db3fdaace092>(232)<module>()
    230                         bloc_i+=1
    231                         #fig.add_subplot(projection='3d')
--> 232                         pca_M0.append(pca[labels==0])
    233                         pca_M1.append(pca[labels==1])
    234                         pca_M2.append(pca[labels==2])

ipdb> n
> <ipython-input-7-db3fdaace092>(233)<module>()
    231                         #fig.add_subplot(projection='3d')
    232                         pca_M0.append(pca[labels==0])
--> 233                         pca_M1.append(pca[labels==1])
    234                         pca_M2.append(pca[labels==2])
    235 

ipd

ipdb> pca_M0
[array([[-2.67800853e+00, -6.10235242e-01,  2.08740531e+00],
       [ 3.59845602e-01,  1.08983537e-01, -3.37825854e-02],
       [-1.17637011e-01, -8.40550469e-02,  6.15803687e-01],
       [-1.67003407e+00, -4.62273850e-01,  1.69362073e+00],
       [ 1.69671930e-01, -2.77616695e-01, -1.02054442e-02],
       [-3.29542999e-01, -9.41085719e-01, -5.05029813e-01],
       [-2.22081828e+00, -4.54340003e+00, -2.59685498e+00],
       [-9.94263523e-01, -8.58655150e-01, -2.68617658e-01],
       [ 2.44063152e-01, -1.98790702e-01,  3.43138253e-01],
       [-7.01340880e-02, -7.19524595e-01, -2.94791230e-01],
       [ 4.04872582e-01,  7.57838433e-03,  1.79897977e-01],
       [ 4.19792400e-01,  9.59581493e-02,  6.28869618e-02],
       [-7.41149702e-02, -3.09325227e-01,  8.45446692e-01],
       [ 6.02496775e-01,  4.37244766e-02, -1.35896516e-01],
       [ 3.30780327e-01, -1.94623491e-01, -3.49794372e-02],
       [ 3.73086682e-01, -1.28613970e-01, -3.87434464e-02],
       [ 4.57176125e-05, -

ipdb> n
> <ipython-input-7-db3fdaace092>(242)<module>()
    240 
    241                     ax.scatter(pca_M0[0][:,0],pca_M0[0][:,1],pca_M0[0][:,2],label='M0_0',c='r',alpha=0.5,zdir='z')
--> 242                     ax.scatter(pca_M1[0][:,0],pca_M1[0][:,1],pca_M1[0][:,2],label='M1_0',c='g',alpha=0.5,zdir='z')
    243                     ax.scatter(pca_M2[0][:,0],pca_M2[0][:,1],pca_M2[0][:,2],label='M2_0',c='b',alpha=0.5,zdir='z')
    244 

ipdb> n
> <ipython-input-7-db3fdaace092>(243)<module>()
    241                     ax.scatter(pca_M0[0][:,0],pca_M0[0][:,1],pca_M0[0][:,2],label='M0_0',c='r',alpha=0.5,zdir='z')
    242                     ax.scatter(pca_M1[0][:,0],pca_M1[0][:,1],pca_M1[0][:,2],label='M1_0',c='g',alpha=0.5,zdir='z')
--> 243                     ax.scatter(pca_M2[0][:,0],pca_M2[0][:,1],pca_M2[0][:,2],label='M2_0',c='b',alpha=0.5,zdir='z')
    244 
    245 

ipdb> n
> <ipython-input-7-db3fdaace092>(247)<module>()
    245 
    246 
--> 247                     ax.scatter

ipdb> n
> <ipython-input-7-db3fdaace092>(249)<module>()
    247                     ax.scatter(pca_M0[1][:,0],pca_M0[1][:,1],pca_M0[1][:,2],label='M0_1',c='y',alpha=0.5,zdir='z')
    248                     ax.scatter(pca_M1[1][:,0],pca_M1[1][:,1],pca_M1[1][:,2],label='M1_1',c='k',alpha=0.5,zdir='z')
--> 249                     ax.scatter(pca_M2[1][:,0],pca_M2[1][:,1],pca_M2[1][:,2],label='M2_1',c='m',alpha=0.5,zdir='z')
    250 
    251                     ax.legend()

ipdb> n
> <ipython-input-7-db3fdaace092>(251)<module>()
    249                     ax.scatter(pca_M2[1][:,0],pca_M2[1][:,1],pca_M2[1][:,2],label='M2_1',c='m',alpha=0.5,zdir='z')
    250 
--> 251                     ax.legend()
    252                     ax.set_title(band_dic[i_band]+' pca projection PC')
    253 

ipdb> n
> <ipython-input-7-db3fdaace092>(252)<module>()
    250 
    251                     ax.legend()
--> 252                     ax.set_title(band_dic[i_band]+' pca projection PC')
    253 
    254      

In [13]:
np.concatenate((pca_M0[0],pca_M1[0],pca_M2[0]),axis=0).mean()


3.2611302667175225e-17

In [14]:
np.concatenate((pca_M0[1],pca_M1[1],pca_M2[1]),axis=0).mean()

-3.1801784848597833e-17

In [15]:
np.concatenate((pca_M0[0],pca_M1[0],pca_M2[0]),axis=0).std()

0.1001134400645769

In [16]:
np.concatenate((pca_M0[1],pca_M1[1],pca_M2[1]),axis=0).std()

0.060229399946049395

In [ ]:
blocs=[[0,6,8,9,4,5],[1,7,2,3,10,11]]

In [12]:
%matplotlib qt

import scipy.io as sio
import os
#from EEGs_persistence import *
from preprocess_data import *
from TDApipeline import *
from intensities_pipe import *
import time  
import sklearn.pipeline as skppl
import sklearn.linear_model as skllm
import sklearn.model_selection as skms
import sklearn.metrics as skm
import matplotlib.pyplot as plt
import sklearn.preprocessing as skprp
import pandas as pd
import sklearn.neighbors as sklnn
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sn
'''
from joblib import Memory
from shutil import rmtree

from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
'''


def define_subject_dir(i_sub):
    """
    Creates the directory if it doesn't exist
    :param i_sub: subject id
    :return: directory path
    """
    res_dir = "results/subject_" + str(i_sub) + "/"
    if not os.path.exists(res_dir):
        print("create directory:", res_dir)
        os.makedirs(res_dir)
    return res_dir

def load_data(i_sub,space='both'):
    """
    Loads data from electrode space, font space 
    or both for a given subject
    :param i_sub: subject id
    :param space: electrode/font_space
    :return: data,directory path
    """
    subj_dir = define_subject_dir(i_sub)
    raw_data = sio.loadmat('data/dataClean-ICA3-'+str(i_sub)+'-T1.mat')
    
    if space=='electrodeSpace':
        elec_space=raw_data['dataSorted'] # [N,T,n_trials,motiv] 
        return elec_space,subj_dir
    elif space=='fontSpace':
        font_space=raw_data['ic_data3']
        return font_space,subj_dir
    else:
        elec_space=raw_data['dataSorted'] # [N,T,n_trials,motiv] 
        font_space=raw_data['ic_data3']
        return (elec_space,font_space),subj_dir,raw_data['indexM']
    


if __name__ == "__main__":
    subjects=[25]
    
    intensities=False
    exploratory=False
    classification=False
    PCA=True
    last=False
    
    bloc_dic={}
    bloc_subj_dic={}
    bloc_subj_dic[27]=np.array([[1, 2, 10, 6, 7, 3],[5, 2, 4, 1, 8, 9]])
    bloc_subj_dic[25]=np.array([[1, 2, 8, 3, 5, 4],[6, 7, 2, 10, 1, 9]])


    
    bands=[-1,0,1,2]  
    #bands=[-1,2]
    n_band=len(bands)
    measures=["euclidean","correlation","quaf"]#,"dtw"]
    #measures=["quaf","dtw"]
    n_measure=len(measures)
    dimensions=["zero","one"]
    #dimensions=[]
    n_dim=len(dimensions)
    feat_vect=[DimensionLandScape(),DimensionSilhouette(),TopologicalDescriptors()]
    
    
    data_table=np.zeros((2*len(subjects),21))
    subj_t=0
    
    n_vectors=len(feat_vect)
    for subject in subjects:
        space='both'
        data_space,subj_dir,index=load_data(subject,space=space)

        spaces=['electrodeSpace','fontSpace']
        index=index[0]

        cont1=0
        cont2=0
        for ind in range(12):
            if index[ind]==1:
                cont1+=1
                if cont1==2:
                    index[ind]=11
            if index[ind]==2:
                cont2+=1
                if cont2==2:
                    index[ind]=12 
        
        bloc_subj_dic[subject][1][bloc_subj_dic[subject][1]<=2]=bloc_subj_dic[subject][1][bloc_subj_dic[subject][1]<=2]+10

        bloc_session=np.where([ind in bloc_subj_dic[subject][1] for ind in index],2,1 )
        
        for sp in range(1):
            t=time.time()
            space=spaces[sp]
            if not os.path.exists(subj_dir+space):
                print("create directory(plot):",subj_dir+space)
                os.makedirs(subj_dir+'/'+space)
            print('cleaning and filtering data of',space,'of subject',subject)
            preprocessor=Preprocessor(data_space[sp])
            #filtered_ts_dic=preprocessor.get_filtered_ts_dic()
            ts_band,labels_original=preprocessor.get_trials_and_labels()
            
            blocs=[]
            blocs.append(np.array(list(range(12)))[bloc_session==1])
            blocs.append(np.array(list(range(12)))[bloc_session==2])
            band_dic={-1: 'noFilter', 0:'alpha',1:'betta',2:'gamma'}
            if PCA: 

                N=ts_band.shape[-1]
                persistence={}
                persistence_2d={}
                for i_band in bands:

                    persistence[i_band]={}
                    persistence_2d[i_band]={}
                    print('global picture of band',band_dic[i_band] )
                    bloc_i=0
                    fig = plt.figure(figsize=[18,8])

                    for bl in blocs:
                        #bloc=ts_band[:,i_band,:,:][temp]
                        PC=np.abs(ts_band[:,i_band,:,:]).mean(axis=1)
                        PC=PC.reshape((-1,N))
                        labels=labels_original
                        PC,labels=preprocessor.reject_outliers(PC,labels)
                        
                        tr2bl=preprocessor.tr2bl
                        temp=[tr_bl in bl for tr_bl in tr2bl]
                        PC=PC[temp]
                        labels=labels[temp]

                        data_table[subj_t,3+i_band]=PC.shape[0]

                        X =(PC - np.mean(PC, axis=0)).T
                        n = X.shape[1]
                        Y =  X.T/np.sqrt(n-1)
                        u, s, vh = la.svd(Y, full_matrices=False)
                        r=np.sum(np.where(s>1e-12,1,0))
                        #pca = vh[:r,:] @ X[:,:] # Principal components
                        variance_prop = s[:r]**2/np.sum(s[:r]**2) # Variance captured
                        acc_variance = np.cumsum(variance_prop)
                        std = s[:r]

                        '''
                        fig, axs = plt.subplots(1, 2, figsize=(18, 4))

                        # 3/4 of the total variance rule
                        axs[0].scatter(range(len(acc_variance)),acc_variance*100)
                        axs[0].set_xticks(range(len(acc_variance)), minor=False)
                        axs[0].hlines(75, xmin=0, xmax=len(std), colors='r', linestyles='dashdot')
                        axs[0].set_title('3/4 of the total variance rule')
                        axs[0].set_xlabel('PCA coordinates')
                        axs[0].set_ylabel('accumulated variance')
                        # Kraiser rule: Keep PC with eigenvalues > 1
                        # Scree plot: keep PCs before elbow
                        axs[1].scatter(range(len(std)),(std**2))
                        axs[1].set_xticks(range(len(acc_variance)), minor=False)

                        axs[1].hlines(1, xmin=0, xmax=len(std), colors='r', linestyles='dashdot')
                        axs[1].set_title('Scree Plot')
                        axs[1].set_xlabel('PCA coordinates')
                        axs[1].set_ylabel('eigenvalue')

                        if not os.path.exists(subj_dir+space+'/PCA/'+band_dic[i_band] ):
                            print("create directory(plot):",subj_dir+space+'/PCA/'+band_dic[i_band] )
                            os.makedirs(subj_dir+space+'/PCA/'+band_dic[i_band] )
                        plt.savefig(subj_dir+space+'/PCA/'+band_dic[i_band]+'/pca_plots.png')
                        plt.close()
                        print('acumulated variance:',acc_variance)'''

                        pca = vh[:3,:] @ X[:,:] 


                        pca=pca.T

                        
                        #fig.add_subplot(projection='3d')
                        pca_M0=pca[labels==0]
                        pca_M1=pca[labels==1]
                        pca_M2=pca[labels==2]

                    

                    
                        ax =fig.add_subplot(1, 2, bloc_i+1, projection='3d')
                        fig.add_axes(ax)
                        
                    
                    
                        ax.scatter(pca_M0[:,0],pca_M0[:,1],pca_M0[:,2],label='M0',c='r',alpha=0.5,zdir='z')
                        ax.scatter(pca_M1[:,0],pca_M1[:,1],pca_M1[:,2],label='M1',c='g',alpha=0.5,zdir='z')
                        ax.scatter(pca_M2[:,0],pca_M2[:,1],pca_M2[:,2],label='M2',c='b',alpha=0.5,zdir='z')
                    
                
                        ax.legend()
                        ax.set_title(band_dic[i_band]+' pca projection PC of session' + str(bloc_i+1))
                        bloc_i=+1

                        ax.set_xlim3d(-1, 1)
                        ax.set_ylim3d(-1, 1)
                        ax.set_zlim3d(-1, 1)

                        ax.set_xlabel('$X$')
                        ax.set_ylabel('$Y$')
                        ax.set_zlabel('$Z$')
                        plt.show()

cleaning and filtering data of electrodeSpace of subject 25
there are 42 clean channels
global picture of band noFilter
global picture of band alpha
global picture of band betta
global picture of band gamma
